In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, glob

In [35]:
path = os.path.join(os.path.dirname(os.getcwd()), '../Data/Extracted_data/')
os.chdir(path)

### MBN

In [63]:
MBN = pd.read_csv('MBN.csv', index_col='Sample')

MBN = MBN.groupby("Sample").mean()

MBN['peak_ratio'] = MBN['peak']/MBN['peak_ref']
MBN['fwhm_ratio'] = MBN['fwhm']/MBN['fwhm_ref']
MBN['diff_two_peaks_ratio'] = MBN['diff_two_peaks']/MBN['diff_two_peaks_ref']
MBN['amp_ratio'] = MBN['amp']/MBN['amp_ref']
MBN['pos_ratio'] = MBN['pos']/MBN['pos_ref']
MBN['energy_ratio'] = MBN['energy']/MBN['energy_ref']

MBN = MBN.drop(columns=['location', 'measure', 
                        'peak', 'peak_ref', 
                        'fwhm', 'fwhm_ref', 
                        'diff_two_peaks', 'diff_two_peaks_ref',
                        'amp', 'amp_ref',
                        'pos', 'pos_ref',
                        'energy', 'energy_ref'])
MBN = MBN[['diff_two_peaks_ratio', 'amp_ratio',
       'pos_ratio', 'energy_ratio']]
print(MBN.shape)
MBN.head()

(36, 4)


,diff_two_peaks_ratio,amp_ratio,pos_ratio,energy_ratio
Sample,,,,
10C,0.230009,1.257148,0.992710,0.457476
13C,1.163056,1.237837,1.002948,1.109720
14C,1.677959,0.939024,0.966947,1.755207
16C,0.684264,0.927139,0.987522,0.910617
19C,0.763296,1.128928,1.002407,1.126849


### NLE

In [1]:
NLE = pd.read_csv('NLE.csv')
NLE['NLE_avg'] = NLE[['S10', 'S11', 'S12']].mean(axis=1)
NLE = pd.pivot_table(NLE, index='Sample' ,columns='freq', values='NLE_avg')
NLE = NLE.add_prefix('NLE_avg_')
NLE[['NLE_ratio_51_17', 'NLE_ratio_85_17', 'NLE_ratio_119_17']] = NLE[['NLE_avg_51', 'NLE_avg_85', 'NLE_avg_119']].div(NLE['NLE_avg_17'], axis=0)
NLE[['NLE_ratio_17_119', 'NLE_ratio_51_119', 'NLE_ratio_85_119']] = NLE[['NLE_avg_17', 'NLE_avg_51', 'NLE_avg_85']].div(NLE['NLE_avg_119'], axis=0)
NLE = NLE[['NLE_ratio_51', 'NLE_ratio_85', 'NLE_ratio_119']]
print(NLE.shape)
NLE.head()

NameError: name 'pd' is not defined

### Electric

In [65]:
electric = pd.read_csv('Electric_RP.csv', index_col='ID')
electric.drop('Unnamed: 0', axis=1, inplace=True)
electric = electric[['Avg_RP', 'ACEE_30_Avg_58', 'ACEE_500_Avg_58', 'ACPD_Avg_58']]
print(electric.shape)
electric.head()

(36, 4)


,Avg_RP,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
ID,,,,
41C,190.979294,-0.137640,-2.866030,4.804749
43C,184.930215,-0.462040,-2.254125,3.869428
44C,187.204261,-0.596385,-2.788500,3.370729
45C,189.010633,-0.645710,-3.202220,4.186639
7C,179.123964,-1.135960,-1.718115,3.438451


### NLO

In [66]:
NLO = pd.read_csv('NLO_result.csv', index_col='Sample')
NLO.drop(['Stress', 'St Dev','Confidence'], axis=1, inplace=True)
NLO.rename(columns={'Average':'NLO_avg'}, inplace=True)
NLO = NLO.groupby('Sample').mean()
print(NLO.shape)
NLO.head()


(36, 1)


,NLO_avg
Sample,
10C,4.240700
13C,9.687991
14C,4.588220
16C,3.755165
19C,3.798736


### Fatigue

In [67]:
fatigue = pd.read_excel('FatigueSampleInventory.xlsx', 
                        usecols=[' Specimen ID', 'Cycles at NDE Test', 'Cycles at Failure', 'Current Unfailed Cycles']
                       )
fatigue.rename(columns={' Specimen ID':'ID', 'Cycles at NDE Test':'NDE_cyle', 'Cycles at Failure':'failure_cycle', 'Current Unfailed Cycles':'unfailed_cycle'}, inplace=True)
fatigue.set_index('ID', inplace=True)
fatigue['f'] = fatigue['NDE_cyle']/fatigue['failure_cycle']
print(fatigue.shape)
fatigue.head()

(36, 4)


,NDE_cyle,failure_cycle,unfailed_cycle,f
ID,,,,
41C,0,2154137.0,NaN,0.0
43C,0,NaN,NaN,NaN
44C,0,1318034.0,NaN,0.0
45C,0,NaN,3245267.0,NaN
7C,150000,NaN,NaN,NaN


Merge all data

In [68]:
merged = fatigue.join([MBN, NLE, NLO, electric], how='left')

In [69]:
merged.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_on_sample.csv'))

In [70]:
merged.sort_index()

,NDE_cyle,failure_cycle,unfailed_cycle,f,diff_two_peaks_ratio,amp_ratio,pos_ratio,energy_ratio,NLE_ratio_51,NLE_ratio_85,NLE_ratio_119,NLO_avg,Avg_RP,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
ID,,,,,,,,,,,,,,,,
10C,300000,NaN,NaN,NaN,0.230009,1.257148,0.992710,0.457476,0.918086,1.125854,0.502010,4.240700,171.622934,-0.797640,-1.795000,2.579638
13C,150000,NaN,NaN,NaN,1.163056,1.237837,1.002948,1.109720,0.919655,1.101159,0.467760,9.687991,170.130491,-0.660875,-2.167395,3.725514
14C,2000000,NaN,NaN,NaN,1.677959,0.939024,0.966947,1.755207,0.903120,1.079935,0.473686,4.588220,159.347604,-0.828250,-0.828250,3.291392
16C,2000000,NaN,NaN,NaN,0.684264,0.927139,0.987522,0.910617,0.900150,1.053618,0.440301,3.755165,169.558497,-1.108970,-1.808550,2.446152
19C,450000,914392.0,NaN,0.492130,0.763296,1.128928,1.002407,1.126849,0.978877,1.235508,0.530102,3.798736,171.925123,-1.065155,-1.570740,3.226957
20C,2000000,NaN,NaN,NaN,0.499761,0.975842,0.995773,0.981373,0.894880,1.078073,0.473923,3.791726,170.021810,-1.223960,-1.223960,2.646820
21C,600000,800007.0,NaN,0.749993,0.705402,1.151889,0.984968,0.853808,0.929446,1.119635,0.493245,4.626314,174.003954,-0.884900,-1.578795,2.764862
23C,600000,NaN,2855225.0,NaN,0.397580,0.919473,0.999861,0.846653,0.918699,1.114762,0.495867,5.621833,173.403495,-0.831110,-1.194540,3.061743
24C,300000,NaN,NaN,NaN,0.588694,1.061223,1.005517,1.030618,0.844482,1.030615,0.438170,5.064456,174.454983,-0.816755,-1.836935,4.259982


In [71]:
merged.shape

(36, 16)